In [1]:
# notebooks/02_features_models.ipynb

import pandas as pd

# Muat data bersih dari master table
PROCESSED_DATA_PATH = "../data/processed/"
df = pd.read_parquet(PROCESSED_DATA_PATH + 'master_table.parquet')

# Pastikan tipe data tanggal sudah benar
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

print("Master table dimuat, jumlah baris:", len(df))

# --- Menghitung RFM ---

# 1. Tentukan tanggal "sekarang" (snapshot_date) sebagai satu hari setelah transaksi terakhir
snapshot_date = df['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
print("Tanggal snapshot untuk perhitungan RFM:", snapshot_date)

# 2. Agregasi data di level pelanggan
customer_data = df.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days, # Recency
    'order_id': 'nunique', # Frequency
    'payment_value': 'sum' # Monetary
})

# 3. Ganti nama kolom agar lebih jelas
customer_data.rename(columns={
    'order_purchase_timestamp': 'recency',
    'order_id': 'frequency',
    'payment_value': 'monetary'
}, inplace=True)

# 4. Membuat skor RFM (misal: dibagi menjadi 4 kuantil)
customer_data['R_score'] = pd.qcut(customer_data['recency'], 4, labels=[4, 3, 2, 1])
customer_data['F_score'] = pd.qcut(customer_data['frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
customer_data['M_score'] = pd.qcut(customer_data['monetary'], 4, labels=[1, 2, 3, 4])

# 5. Gabungkan skor menjadi RFM Score
customer_data['RFM_score'] = customer_data['R_score'].astype(str) + customer_data['F_score'].astype(str) + customer_data['M_score'].astype(str)

# Tampilkan hasilnya
customer_data.head()

Master table dimuat, jumlah baris: 117329
Tanggal snapshot untuk perhitungan RFM: 2018-09-04 09:06:57


,recency,frequency,monetary,R_score,F_score,M_score,RFM_score
customer_unique_id,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,116,1,141.90,4,1,3,413
0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,27.19,4,1,1,411
0000f46a3911fa3c0805444483337064,542,1,86.22,1,1,2,112
0000f6ccb0745a6a4b88665a16c9f078,326,1,43.62,2,1,1,211
0004aac84e0df4da2b147fca70cf8255,293,1,196.89,2,1,3,213
